In [ ]:
!pip install -q python-dotenv 

In [1]:
from dotenv import load_dotenv

import json
import os
import pandas as pd
import requests

# Environment variables

Get a Datawrapper API token: https://app.datawrapper.de/account/api-tokens

In [2]:
beer_data_filename = 'beer_data.csv'

# Load variables in .env file.
load_dotenv()

DATAWRAPPER_TOKEN = os.getenv('DATAWRAPPER_TOKEN')

# Load CSV Beer list

In [3]:
beer_pd = pd.read_csv(beer_data_filename)
beer_fil_pd = beer_pd.dropna(subset=['Beer'])
beer_fil_pd = beer_fil_pd.drop(['Beer', 'Date', 'Month'], axis=1).drop_duplicates()

In [4]:
beer_loc = beer_fil_pd.dropna(subset=['lng'])
beer_loc = beer_loc[beer_loc['lng'] != 0]
len(beer_loc)

127

# Statistics

In [16]:
brewery_country_pd = beer_fil_pd.groupby('brewery_city')['beer_name'].count().reset_index().rename(columns={'beer_name': 'count'})\
    .sort_values('count', ascending=False).head(10)
brewery_country_pd['percentage'] = (brewery_country_pd['count'] / brewery_country_pd['count'].sum())*100
brewery_country_pd

,brewery_city,count,percentage
3,Amsterdam,14,24.561404
52,Utrecht,9,15.789474
24,Haarlem,9,15.789474
23,Gulpen,5,8.771930
35,Leuven,5,8.771930
1,Achouffe,3,5.263158
26,Hilversum,3,5.263158
11,Bruges,3,5.263158
38,Mechelen,3,5.263158
7,Berkel-Enschot,3,5.263158


In [14]:
brewery_state_pd = beer_fil_pd.groupby('brewery_state')['beer_name'].count().reset_index().rename(columns={'beer_name': 'count'})\
    .sort_values('count', ascending=False).head(10)
brewery_state_pd['percentage'] = (brewery_state_pd['count'] / brewery_state_pd['count'].sum())*100
brewery_state_pd

,brewery_state,count
14,Noord-Holland,31
21,Vlaanderen,30
19,Utrecht,10
22,Wallonie,9
10,Limburg,8
13,Noord-Brabant,5
24,Zuid-Holland,5
7,Gelderland,3
17,Overijssel,2
15,Nordrhein-Westfalen,2


In [19]:
brewery_country_pd = beer_fil_pd.groupby('country_name')['beer_name'].count().reset_index().rename(columns={'beer_name': 'count'})\
    .sort_values('count', ascending=False).head(10)
brewery_country_pd['percentage'] = (brewery_country_pd['count'] / brewery_country_pd['count'].sum())*100
brewery_country_pd

,country_name,count,percentage
4,Netherlands,69,53.90625
0,Belgium,42,32.81250
1,Germany,5,3.90625
9,United States,4,3.12500
2,Greece,2,1.56250
7,Scotland,2,1.56250
3,Ireland,1,0.78125
5,Poland,1,0.78125
6,Portugal,1,0.78125
8,Spain,1,0.78125


# Datawrapper

Creating a locator map: https://developer.datawrapper.de/docs/creating-a-locator-map

## Create map

In [ ]:
dw_url = "https://api.datawrapper.de/v3/charts"
dw_headers = {
    'Authorization': f'Bearer {DATAWRAPPER_TOKEN}',
    'content-type': 'application/json'
}
data_create = {
    "title": "Beer Club", 
    "type": "locator-map"
}

response = requests.post(dw_url, headers=dw_headers, json=data_create)
chart_id = response.json()['id']

## Place pin

In [ ]:
dw_url = f"https://api.datawrapper.de/v3/charts/{chart_id}/data"
dw_headers = {
    'Authorization': f'Bearer {DATAWRAPPER_TOKEN}',
    'content-type': 'text/csv'
}

markers = []

for _, row in beer_loc.iterrows():
    markers.append({
            "type": "point",
            "title": row['brewery_name'],
            "tooltip": {
                "enabled": False, 
                "text": "Brewery: {0}\nAlcoholpercentage: {1}".format(row['brewery_name'], row['beer_abv'])},
            "icon": {
                "id": "locator",
                "path": "M714 487a367 367 0 0 0-32-151c-56-125-325-486-325-486s-268 361-325 486a367 367 0 0 0-32 151 360 360 0 0 0 357 363 360 360 0 0 0 357-363z",
                "horiz-adv-x": 714,
                "vCenter": 0,
                "textBaseline": 475,
                "scale": 1.3,
                "height": 734.1012497777449,
                "width": 713.9999999999997},
            "scale": 1,
            "markerColor": "#cc0000",
            "anchor": "middle-right",
            "offsetY": 0,
            "offsetX": 0,
            "text": {
            "color": "#333333",
            "fontSize": 17,
            "halo": "#f2f3f0"
            },
            "rotate": 0,
            "visible": True,
            "visibility": {
            "mobile": True,
            "desktop": True
            },
            "coordinates": [
                row['lng'], 
                row['lat']
            ]
        })
    
dw_json = { "markers": markers }

requests.put(dw_url, headers=dw_headers, json=dw_json)

## Tweak description, zoom and other details

In [ ]:
dw_url = f'https://api.datawrapper.de/v3/charts/{chart_id}'
dw_headers = {
    'Authorization': f'Bearer {DATAWRAPPER_TOKEN}',
    'content-type': 'application/json'
}
dw_data = {
    "metadata": {
        "describe": {
              "intro": "All beers from the Beer Club",
              "byline": "Data Intelligence, RTL"
          }
    }
}

response = requests.patch(dw_url, headers=dw_headers, json=dw_data)

In [ ]:
dw_url = f'https://api.datawrapper.de/v3/charts/{chart_id}'
dw_headers = {
    'Authorization': f'Bearer {DATAWRAPPER_TOKEN}',
    'content-type': 'application/json'
}

# Western World
dw_data = {
    "metadata": {
        "visualize": {
            "view": {
                "bbox": [
                    [-138, -9],
                    [60, 68]
                ],
                "zoom": 1,
                "pitch": 0,
                "center": [-37, 40],
                'height': 50
            }
        }
    }
}
# Benelux
dw_data = {
    "metadata": {
        "visualize": {
               'view': {'bbox': [[1.9935443395971788, 50.58007704402837],
     [8.114292247008905, 53.43626246536874]],
    'zoom': 6,
    'pitch': 0,
    'center': [5.05391829330506, 52.0309679816439],
    'height': 75,
    'bearing': 1},
#             "view": {
#                 "bbox": [
#                     [2, 50],
#                     [10, 54]
#                 ],
#                 "zoom": 1,
#                 "pitch": 0,
#                 "center": [6, 52],
#                 'height': 75,
#                 'bearing': 1
#             },
            'style': 'dw-earth',
            'visibility': {
                'green': True,
                'roads': True,
                'urban': True,
                'water': True,
                'building': True,
                'mountains': False,
                'building3d': False,
                'boundary_state': False,
                'boundary_country': True
            }
        }
    }
}

response = requests.patch(dw_url, headers=dw_headers, json=dw_data)

## Publish chart

In [ ]:
dw_url = f'https://api.datawrapper.de/v3/charts/{chart_id}/publish'
dw_headers = {'Authorization': f'Bearer {DATAWRAPPER_TOKEN}'}

response = requests.post(dw_url, headers=dw_headers)

In [ ]:
# Get properties published map
dw_url = f'https://api.datawrapper.de/v3/charts/{chart_id}'
dw_headers = {'Authorization': f'Bearer {DATAWRAPPER_TOKEN}'}

response = requests.get(dw_url, headers=dw_headers)